In [1]:
from transformers import DonutProcessor
import sys
sys.path.insert(1, 'PubTabNet/src/')
sys.path.insert(1, '../proj/src/')

from metric import TEDS
from processing_tabeleiro import TabeleiroProcessor
import json
import os


SPLIT = "val/"

PUBTABNET = 'data/anns/'

ANN_PATH = PUBTABNET

json_list = os.listdir(ANN_PATH + SPLIT)

aux_list = []
html_list = []

for f in json_list:
    if(f[-6:-5] == "L"):
        html_list.append(f)
    else:
        aux_list.append(f)
json_list = aux_list
processor = TabeleiroProcessor.from_pretrained("processors/Donut_PubTables_TML_Processor8k", crop_right=False)

/home/jao/Documents/UFLA/Mestrado/venv_mestrado/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
json_list[:10]

['PMC5771025_005_00.json',
 'PMC4077014_004_00.json',
 'PMC4042280_008_01.json',
 'PMC5629611_006_00.json',
 'PMC4824228_005_00.json',
 'PMC6060525_003_00.json',
 'PMC2265025_002_00.json',
 'PMC4846708_004_00.json',
 'PMC4100296_012_00.json',
 'PMC3112163_003_00.json']

In [3]:
gt_list = []
pred_list = []

teds = TEDS(n_jobs=4, structure_only = False)

In [4]:
processor.tokenizer("</s>")

{'input_ids': [0, 2, 2], 'attention_mask': [1, 1, 1]}

In [ ]:
import random
import torch
from difflib import SequenceMatcher
from tqdm.auto import tqdm

removed_tags = []
soma_ratios = 0

for i, file_name in enumerate(tqdm(json_list)):

    with open(ANN_PATH + SPLIT + file_name) as f:
        annotation = json.load(f)
    with open(ANN_PATH + SPLIT + file_name[:-5]+"-HTML.json", encoding="utf-8") as f:
        gt_html = "<html><body><table>"+str(json.load(f))+"</table></body></html>"
    
    for tag in removed_tags:
        gt_html = gt_html.replace(tag, "")
    
    target_sequence = "<s>"+processor.json2token(annotation)+"</s>"
    input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=False,
        max_length= 2048,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)
    
    
    input_ids = torch.cat((input_ids, torch.Tensor([2, 2]).int()), 0)
    
    table = processor.token2ann( input_ids, 1)
    table_aux  = table.copy()
    
    
    pred_html = "<html><body><table>"+processor.table2html(table['tables'][0])+"</table></body></html>"
    
    pred_html = pred_html
    gt_html = gt_html.replace(" </td>", "</td>").replace("<td> ", "<td>").replace('"> ', '">')
    pred_list.append(pred_html)
    gt_list.append(gt_html)
    
    ratio = teds.evaluate(str(pred_html), str(gt_html))
    
    if(ratio < .9):
        print(processor.decode(input_ids[1]))
        #print(target_sequence)
        #print(table['tables'][0])
        print(pred_html)
        print()
        print(gt_html)
        print(ratio)
    if i% 100 == 0:   
        print(str(i)+'->', soma_ratios/max(1, i))
    
    soma_ratios += ratio
print(soma_ratios/len(json_list))

  0%|                                          | 1/9116 [00:00<54:57,  2.76it/s]

0-> 0.0


  0%|▏                                      | 38/9116 [00:58<7:34:48,  3.01s/it]

<table>
<html><body><table><thead><tr><td><b>Exposure</b></td><td><b>1st Trimester</b></td><td><b>2nd Trimester</b></td><td><b>3rd Trimester</b></td></tr></thead><tbody><tr><td><b>Spatiotemporally resolved PM<sub>2.5</sub></b></td><td></td><td></td><td></td></tr><tr><td><b><i>n</i></b></td><td>765</td><td>845</td><td>970</td></tr><tr><td><b>IQR (μg/m<sup>3</sup>)</b></td><td>2.29</td><td>1.97</td><td>2.24</td></tr><tr><td><b>β (95% CI)</b></td><td>–0.3 (–1.3, 0.7)</td><td>0.4 (–0.4, 1.2)</td><td>0.3 (–0.6, 1.2)</td></tr><tr><td><b>Temporally resolved PM<sub>2.5</sub></b></td><td></td><td></td><td></td></tr><tr><td><b><i>n</i></b></td><td>1,032</td><td>1,031</td><td>1,030</td></tr><tr><td><b>IQR (μg/m<sup>3</sup>)</b></td><td>2.35</td><td>1.77</td><td>2.05</td></tr><tr><td><b>β (95% CI)</b></td><td>0.1 (–0.9, 1.2)</td><td>0.1 (–0.7, 1.0)</td><td>0.5 (–0.4, 1.5)</td></tr><tr><td><b>Spatiotemporally resolved BC</b></td><td></td><td></td><td></td></tr><tr><td><b><i>n</i></b></td><td>1,099<

  0%|▏                                      | 40/9116 [00:59<4:35:27,  1.82s/it]

In [ ]:
json.dump(pred_list, open("teste-pred.json", 'w'))
json.dump(gt_list, open("teste-gt.json", 'w'))

In [ ]:
a = '<thead><tr><td rowspan="2"><b>Cutoff (IRS)</b></td><td rowspan="2"><b>Number</b></td><td colspan="2"><b>Overall survival prognosis</b></td></tr><tr><td><b>Log-rank <i>P</i></b></td><td><b>Corrected <i>P</i><sup><i>∗</i></sup></b></td></tr></thead><tbody><tr><td>≥1</td><td>99/94</td><td>5.56</td><td>0.0184</td></tr><tr><td>≥2</td><td>129/64</td><td>4.37</td><td>0.0377</td></tr><tr><td>≥3</td><td>150/43</td><td>8.37</td><td>0.0038</td></tr><tr><td>≥4</td><td>161/32</td><td>6.35</td><td>0.0117</td></tr><tr><td>≥5 </td><td>177/16</td><td>4.50</td><td>0.0338</td></tr></tbody>'

for tag in ["<i>", "</i>","<b>","</b>"]:
    a = a.replace(tag, "")
    
a

In [ ]:
target_sequence = "Capítulo Primeiro Do título Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no[ 1 ] trem da Central um rapaz aqui do bairro, que eu conheço de vista e de cha- péu. Comprimentou-me, sentou-se ao pé de mim, falou da lua e dos mi- nistros, e acabou recitando-me versos. A viagem era curta, e os versos pode ser que não fossem inteiramente maus. Succedeu, porém, que, como eu es- tava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso. — Continue, disse eu acordando.[ 2 ] — Já acabei, murmurou ele.[ 3 ] — São muito bonitos.[ 4 ] Vi-lhe fazer um gesto para tirá-los outra vez do bolso, mas não passou[ 5 ] do gesto; estava amuado. No dia seguinte entrou a dizer de mim nomes feios, e acabou alcunhando-me Dom Casmurro."

input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=True,
        max_length= 1536,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)

In [ ]:
input_ids

In [ ]:
for token in input_ids:
    print(processor.decode(token))

In [ ]:
processor.decode(input_ids)